# sklearn: LogisticRegression

In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)

In [23]:
lr_ovr = LogisticRegression(multi_class='ovr')
lr_ovr.fit(X=X, y=y)
print(lr_ovr.intercept_)
print(lr_ovr.coef_)


[  6.69040651   5.58615272 -14.43121671]
[[-0.44501376  0.89999242 -2.32353827 -0.97345836]
 [-0.1792787  -2.12866718  0.69665417 -1.27480129]
 [-0.39444787 -0.5133412   2.93087523  2.41709879]]


In [9]:
print(lr_ovr.decision_function(X)[:5, 0])
print(lr_ovr.predict_proba(X)[:5, 0])

[4.12316458 3.76217112 4.26352618 3.75332066 4.25766519]
[0.89680757 0.77897939 0.83486418 0.79000199 0.9120504 ]


In [22]:
import numpy as np

# manual preds on linear predictor scale
def predict_link_manual(features, model):
    X = features
    alpha = model.intercept_
    beta = model.coef_
    assert beta.shape[0] == len(alpha)
    assert beta.shape[1] == X.shape[1]
    preds_list = [alpha[i] + np.matmul(X, beta[i, :])
                  for i in range(len(alpha))]
    return np.array(preds_list).T


# manual preds on probability scale
def softmax(preds):
    inv_logit_preds_T = 1 / (1 + np.exp(-preds.T))
    probs = inv_logit_preds_T / sum(inv_logit_preds_T)
    return probs.T


print(predict_link_manual(X, lr_ovr)[:5, 0])
print(softmax(predict_link_manual(X, lr_ovr))[:5, 0])

[4.12316458 3.76217112 4.26352618 3.75332066 4.25766519]
[0.89680757 0.77897939 0.83486418 0.79000199 0.9120504 ]


## StandardScaler + GridSearchCV

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print("X_train shape: {}".format(X_train.shape)) 
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape)) 
print("y_test shape: {}".format(y_test.shape))

lr = LogisticRegression(
    multi_class='ovr',
    penalty='l2',
    intercept_scaling=1, 
    dual=False, 
    fit_intercept=True,
    tol=0.0001,
)

steps = [
   ('scaler', StandardScaler()),
    ('lr', lr),
]

pipeline = Pipeline(steps)

param_grid = [
#    {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
#    {'solver': ['newton-cg', 'liblinear', 'sag', 'saga']},
#    {'penalty': ['l1','l2']}, 
    {'lr__C': [0.01, 0.1, 1, 10, 100, 1000]},
]

cv = GridSearchCV(
    estimator=pipeline,  
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    refit=True,
    verbose=0,
    n_jobs=1,
)

cv.fit(X_train, y_train)

print(cv.best_params_)
clf = cv.best_estimator_
print(clf)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test)) # *
y_pred = clf.predict(X_test)
print(precision_score(y_test, y_pred, average=None)) # precision for each class
print(accuracy_score(y_test, y_pred))  # same as *

X_train shape: (75, 4)
y_train shape: (75,)
X_test shape: (75, 4)
y_test shape: (75,)
{'lr__C': 10}
Pipeline(steps=[('lr', LogisticRegression(C=10, multi_class='ovr'))])
0.96
0.9866666666666667
[1.         0.95833333 1.        ]
0.9866666666666667
